In [1]:
import os

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import torch

from data.util.dataset import MaskedTimeseries
from inference.forecaster import TotoForecaster
from model.toto import Toto

# These lines make gpu execution in CUDA deterministic
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.use_deterministic_algorithms(True)


/Users/ozymandias/work/frc-agent/lib/python3.13/site-packages/lightning/fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
/Users/ozymandias/work/frc-agent/lib/python3.13/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
pip install -r ../requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [4]:
ETT_LOCAL_PATH = "/Users/ozymandias/work/frc-agent/ETDataset/ETT-small"

In [5]:

df = (
    pd.read_csv(f"{ETT_LOCAL_PATH}/ETTm1.csv")
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .assign(timestamp_seconds=lambda df: (df.date - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))
)

df

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,timestamp_seconds
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,1467331200
1,2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999,1467332100
2,2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000,1467333000
3,2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000,1467333900
4,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001,1467334800
...,...,...,...,...,...,...,...,...,...
69675,2018-06-26 18:45:00,9.310,3.550,5.437,1.670,3.868,1.462,9.567000,1530038700
69676,2018-06-26 19:00:00,10.114,3.550,6.183,1.564,3.716,1.462,9.567000,1530039600
69677,2018-06-26 19:15:00,10.784,3.349,7.000,1.635,3.746,1.432,9.426000,1530040500
69678,2018-06-26 19:30:00,11.655,3.617,7.533,1.706,4.173,1.523,9.426000,1530041400


In [6]:


context_length = 4096
prediction_length = 336



In [7]:
feature_columns = ["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"]
n_variates = len(feature_columns)
interval = 60 * 15  # 15-min intervals
input_df = df.iloc[-(context_length+prediction_length):-prediction_length]
target_df = df.iloc[-prediction_length:]
DEVICE = "cuda"

input_series = torch.from_numpy(input_df[feature_columns].values.T).to(torch.float).to(DEVICE)
input_series.shape

AssertionError: Torch not compiled with CUDA enabled